<a href="https://colab.research.google.com/github/C3Suryansu/Odia-writing-recognition-Webapp/blob/main/kfold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install wandb -qqq
# import wandb
# from wandb.keras import WandbCallback
# wandb.login()

# run = wandb.init(project='onr')
# config = wandb.config
# config.learning_rate = 0.001
# config.batch_size = 64
# config.validation_split = 0.2
# config.target_size = (32, 32)
# config.random_state = 42

In [2]:
from google.colab import drive 
drive.mount('drive/', force_remount = True)

Mounted at drive/


In [3]:
!mkdir data
!unrar x "/content/drive/MyDrive/Odia Numericals/IITBBS Odia Numeral DATABASE.rar" -d data

Streaming output truncated to the last 5000 lines.
Extracting  data/9/428.jpg                                                 3%  OK 
Extracting  data/9/429.jpg                                                 3%  OK 
Extracting  data/9/43.jpg                                                  3%  OK 
Extracting  data/9/430.jpg                                                 3%  OK 
Extracting  data/9/432.jpg                                                 3%  OK 
Extracting  data/9/433.jpg                                                 3%  OK 
Extracting  data/9/434.jpg                                                 3%  OK 
Extracting  data/9/435.jpg                                                 3%  OK 
Extracting  data/9/436.jpg                                                 3%  OK 
Extracting  data/9/437.jpg                                                 3%  OK 
Extracting  data/9/438.jpg   

In [12]:
import sys
import os 
import pandas as pd
data_dir = "data/"
num_of_classes = len(os.listdir(data_dir))
print("Classes:",num_of_classes)


# For undersampling data
images = []
labels = []
min_files = sys.maxsize
for label in os.listdir(data_dir):
    files = os.listdir(os.path.join(data_dir, label))
    min_files = min(len(files), min_files)
    for file in files:
        images.append(os.path.join(data_dir,label,file))
        labels.append(label)
    
print(min_files)

df = pd.DataFrame({"images": images, "labels":labels})
df = df.groupby('labels').head(min_files).reset_index(drop=True)
df = df.sample(frac=1).reset_index(drop=True)

Classes: 10
500


# KF using df

In [13]:
import cv2
err_ind = []
for i in range(5000):
  try:
    img = cv2.imread(df.iloc[i]['images'])
    sh = img.shape
  except:
    err_ind.append(i)

In [14]:
df = df.drop(labels = err_ind, axis = 0)
df.head()

,images,labels
0,data/7/205.jpg,7
1,data/5/108.jpg,5
2,data/4/15.jpg,4
3,data/6/6_356.jpg,6
4,data/4/155.jpg,4


In [15]:
df = df.sample(frac=1).reset_index(drop=True)

In [16]:
n = df.shape[0]
n

4997

In [17]:
!mkdir newData

mkdir: cannot create directory ‘newData’: File exists


In [18]:
new_img_names = []
for i in range(n):
  img = cv2.imread(df.iloc[i]['images'])
  cv2.imwrite('newData/' + str(i) + '.jpg', img)
  new_img_names.append(str(i) + '.jpg')

In [19]:
df['images'] = new_img_names

In [20]:
df.head()

,images,labels
0,0.jpg,2
1,1.jpg,7
2,2.jpg,1
3,3.jpg,2
4,4.jpg,8


In [25]:
df.to_csv('/content/data.csv', index = None)
newdf = pd.read_csv('/content/data.csv')
newdf.head()

,images,labels
0,0.jpg,2
1,1.jpg,7
2,2.jpg,1
3,3.jpg,2
4,4.jpg,8


In [26]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, AveragePooling2D
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import activations
import tensorflow as tf
def create_model():
  vggmodel = VGG16(weights='imagenet', include_top = False, input_shape = (32, 32, 3),pooling='max')
  for layer in vggmodel.layers[0:15]:
    layer.trainable = False
  x = vggmodel.output
  x = Dense(64, activation='relu')(x)
  x = Dropout(0.5)(x)
  x = Dense(10, activation='softmax')(x)

  model = models.Model(inputs = vggmodel.input, outputs = x)
  optimizer=tf.keras.optimizers.Adam(learning_rate = 0.001)
  
  model.compile(loss='categorical_crossentropy',
                optimizer = optimizer,
                metrics=['accuracy'])
  return model

In [33]:
from sklearn.model_selection import KFold

kf = KFold(5, shuffle = True, random_state = 42)

In [34]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=3, min_lr=1e-7, verbose=1)
import numpy as np

In [35]:
# img_list = np.array(df['images'])
# x = []
# for i in img_list:
#   img = cv2.imread('newData/' + i)
#   x.append(img)
# x = np.array(x)
# x.shape

In [36]:
# lab = np.array([int(i) for i in df['labels']])
# y = np.zeros((lab.size, lab.max()+1))
# y[np.arange(lab.size), lab] = 1
# y

In [37]:
x = np.array(df['images'])
y = np.array(df['labels'])

In [39]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
oos_y = []
oos_pred = []
train_acc_arr = []
test_acc_arr = []
fold = 0
for train, test in kf.split([i for i in range(4997)]):
    fold+=1
    print(f"Fold #{fold}")
        
    x_train = x[train]
    y_train = y[train]
    x_test = x[test]
    y_test = y[test]
    train_df = pd.DataFrame({'images':x_train, 'labels':y_train})
    test_df = pd.DataFrame({'images':x_test, 'labels':y_test})
    # print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
    # print(train_df.head())
    # print(test_df.head())
    train_image_data = ImageDataGenerator(rescale = 1./255,
                                          shear_range = 0.1,
                                          zoom_range = 0.2)
    # val_image_data = ImageDataGenerator(rescale = 1./255)
    train_generator = train_image_data.flow_from_dataframe(dataframe = train_df,
                                                          directory = 'newData',
                                                          x_col = 'images',
                                                          y_col = 'labels',
                                                          batch_size = 32,
                                                          class_mode = 'categorical',
                                                          target_size = (32, 32))
    test_generator = train_image_data.flow_from_dataframe(dataframe = test_df,
                                                        directory = 'newData',
                                                        x_col = 'images',
                                                        y_col = 'labels',
                                                        batch_size = 32,
                                                        class_mode = 'categorical',
                                                        target_size = (32, 32))
    model = create_model()
    history = model.fit(train_generator,
              validation_data = test_generator,
              verbose = 1,
              epochs = 50, 
              callbacks = [reduce_lr])
    
    # pred = model.predict(x_test) 
    # oos_y.append(y_test)
    # oos_pred.append(pred)    
    train_acc_arr.append(history.history['accuracy'][-1] * 100)
    test_acc_arr.append(history.history['val_accuracy'][-1] * 100)
    # print("Training Accuracy: {} Testing Accuracy: {}".format(history.history['accuracy'][-1] * 100, history.history['val_accuracy'][-1] * 100))
    # print("Training Loss: {} Testing Loss: {}".format(history.history['loss'][-1], history.history['val_loss'][-1]))

#     # Measure this fold's RMSE
#     score = np.sqrt(metrics.mean_squared_error(pred,y_test))
#     print(f"Fold score (RMSE): {score}")

# # Build the oos prediction list and calculate the error.
# oos_y = np.concatenate(oos_y)
# oos_pred = np.concatenate(oos_pred)
# score = np.sqrt(metrics.mean_squared_error(oos_pred,oos_y))
# print(f"Final, out of sample score (RMSE): {score}")    
    
# Write the cross-validated prediction
# oos_y = pd.DataFrame(oos_y)
# oos_pred = pd.DataFrame(oos_pred)
# oosDF = pd.concat( [df, oos_y, oos_pred],axis=1 )
#oosDF.to_csv(filename_write,index=False)

Fold #1
Found 3997 validated image filenames belonging to 10 classes.
Found 1000 validated image filenames belonging to 10 classes.
Epoch 1/50
125/125 [==============================] - 8s 58ms/step - loss: 2.2070 - accuracy: 0.1546 - val_loss: 2.0115 - val_accuracy: 0.1780
Epoch 2/50
125/125 [==============================] - 7s 58ms/step - loss: 1.6246 - accuracy: 0.3710 - val_loss: 1.3029 - val_accuracy: 0.4750
Epoch 3/50
125/125 [==============================] - 7s 57ms/step - loss: 1.0087 - accuracy: 0.6305 - val_loss: 0.3934 - val_accuracy: 0.8430
Epoch 4/50
125/125 [==============================] - 7s 55ms/step - loss: 0.4843 - accuracy: 0.8459 - val_loss: 0.2845 - val_accuracy: 0.9080
Epoch 5/50
125/125 [==============================] - 7s 54ms/step - loss: 0.3173 - accuracy: 0.9042 - val_loss: 0.2079 - val_accuracy: 0.9350
Epoch 6/50
125/125 [==============================] - 7s 55ms/step - loss: 0.2569 - accuracy: 0.9257 - val_loss: 0.1822 - val_accuracy: 0.9480
Epoch 7/50

In [51]:
for i in range(5):
  print(f"Fold #{i + 1}")
  print("Training Accuracy: {:.2f} Testing Accuracy: {:.2f}".format(train_acc_arr[i], test_acc_arr[i]))
  print()

print("Average Training Accuracy: {:.2f} \nAverage Testing Accuracy: {:.2f}".format(sum(train_acc_arr)/5, sum(test_acc_arr)/5))

Fold #1
Training Accuracy: 97.87 Testing Accuracy: 97.10

Fold #2
Training Accuracy: 98.62 Testing Accuracy: 96.80

Fold #3
Training Accuracy: 98.90 Testing Accuracy: 96.60

Fold #4
Training Accuracy: 98.22 Testing Accuracy: 96.90

Fold #5
Training Accuracy: 98.40 Testing Accuracy: 95.90

Average Training Accuracy: 98.40 
Average Testing Accuracy: 96.66


In [30]:
train_df.head()

,images,labels
0,0.jpg,2
1,1.jpg,5
2,2.jpg,8
3,6.jpg,5
4,7.jpg,4


In [31]:
test_df.head()

,images,labels
0,3.jpg,2
1,4.jpg,2
2,5.jpg,1
3,9.jpg,5
4,16.jpg,7
